# Classification of data from a particular news collection into categories such as international news and travel using machine learning and deep learning. Short description of the news dataset is used for the machine learning with category as a target.

Importing the required Statements

In [ ]:
from sklearn.model_selection import cross_val_score
import pandas as pandasForSortingCSV
import nltk
import re
import string
import matplotlib
import numpy as np
from nltk import *
from nltk.corpus import stopwords
import pandas as pd
import matplotlib.pyplot as plt
import gensim
from sklearn.linear_model import LogisticRegressionCV
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import pickle
from pandas import DataFrame as df
from datetime import datetime
from IPython.display import display_html 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer, roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import pandas as pd
import re
import keras.layers as layers
from collections import Counter
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.layers import Input, Embedding, BatchNormalization, LSTM, Dense, Concatenate , Dropout
from keras.models import Model
from keras.utils import plot_model
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import copy
import numpy as np                                                               
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from keras import optimizers
from keras import regularizers

 ##  Task 1

### 2.Loading  my dataset

In [ ]:
mydata=pd.read_csv('21204205.csv', encoding='Latin1',dtype=str)

Editing my dataset with preffered column names

In [ ]:
mydata = mydata.set_axis(["id","category","headline","authors","link","short_description","date"],axis=1)

Viewing the dataset

In [ ]:
mydata

## 3. iii) Removing the nan values and duplicates from the dataset since its a common occurence for the data to have duplicate values and empty values

In [ ]:
mydata=mydata.dropna()
mydata=mydata.drop_duplicates()
mydata

Checking whether there is any empty rows

In [ ]:
mydata.isnull().sum()

## Merging short description and headline into short description

In [ ]:
mydata["short_description"]=mydata['headline']+mydata['short_description']

### Preprocessing :Converting the whole dataset into lowercase

In [ ]:
mydata['short_description'] = mydata['short_description'].str.lower()
mydata['category'] = mydata['category'].str.lower()
mydata['headline'] =[word.lower() for word in mydata['headline']]
mydata['authors'] = [word.lower() for word in mydata['authors']]
mydata['link'] = [word.lower() for word in mydata['link']]

# Plotting the news data set as per category

In [ ]:
mydata['category'].value_counts().plot(kind='bar')

##  After plotting the dataset in relation to category we noticed there is a imbalance and not equally distributed which may lead to predicting the best accuracy scores in travel category and fail to predict exact accuracy in world news category 

Separating the dataset as per the category "travel" and "world news"

In [ ]:
mydata_travel=mydata[mydata["category"] == 'travel']
mydata_news=mydata[mydata["category"] == 'world news']

## Considering the short description of the news dataset,Converting the string to a list 

In [ ]:
# mydata_travel["short_description"].values.tolist()
# mydata_news["short_description"].values.tolist()

Defining Method to convert list to String

In [ ]:
#Method to convert list to String
def listToString(s):
    str1=" "
    return(str1.join(s))

### Preprocessing the data using nltk methods (Tokenization,punctuation and stop words removal)

In [ ]:
mydata_travel_token = nltk.word_tokenize(str(mydata_travel['short_description'].values.tolist()))
mydata_news_token= nltk.word_tokenize(str(mydata_news['short_description'].values.tolist()))
mydata_travel_no_punct = [t for t in mydata_travel_token if t.isalnum()]
mydata_news_no_punct=[t for t in mydata_news_token if t.isalnum()]
mydata_travel_no_stopwords = [t for t in mydata_travel_no_punct if t not in stopwords.words('english')] 
mydata_news_no_stopwords= [t for t in mydata_news_no_punct if t not in stopwords.words('english')]

## 3.i)  Using Frequency distrubution finding the most common words in travel category

In [ ]:
fd_travel = nltk.FreqDist(mydata_travel_no_stopwords)
print(fd_travel.most_common(10))

## 3.i)  Using Frequency distrubution finding the most common words in news category

In [ ]:
fd_news = nltk.FreqDist(mydata_news_no_stopwords)
print(fd_news.most_common(10))

### 3.ii) Taking a copy of data to run feature selection using Chi-Square Test

In [ ]:
# code reference: github.com

DATAAA = pd.DataFrame(mydata)
#Make a copy:
featureselection = DATAAA.copy()
print(featureselection)

In [ ]:
del featureselection["date"]
del featureselection["headline"]
del featureselection["authors"]
del featureselection["link"]

##  Visualization of category using chi-square test by feature selection. Since chi-square test  is measured on a nominal (categorical) scale.

In [ ]:
contigency= pd.crosstab(featureselection['short_description'], featureselection['category'])
contigency_pct = pd.crosstab(featureselection['short_description'], featureselection['category'], normalize='index')
plt.figure(figsize=(12,8))
sns.heatmap(contigency, annot=True, cmap="YlGnBu")

### 3.iii) Length of sentences
Defining a function to find length of each sentence

In [ ]:
def sentence_length(text):
    sentences = text.split(",")
    words = text.split(" ")
    if(sentences[len(sentences)-1]==""):
       sentence_length = len(words)/len(sentences)-1
    else:
       sentence_length = len(words)/len(sentences)
    return sentence_length
#https://python-forum.io/thread-11585.html

In [ ]:
travel_avg=sentence_length(str(mydata_travel['short_description'].values.tolist()))
news_avg=sentence_length(str(mydata_news['short_description'].values.tolist()))

Average of sentences in category "travel"

In [ ]:
print(travel_avg)

Average of sentences in category "world news"

In [ ]:
print(news_avg)

Viewing the columns of our data set

In [ ]:
mydata.columns

### Loaded the dataset and preprocessed by converting into lowercase,removing stop words,special characters to reduce the noice in the dataset, in addition to that we have removed duplicates and empty values. Found the most frequently used words in the short description of both world news and travel category . Used Chi-square test to display feature selection in relation with categories in the task 1


#  Task 2


# Encoding the Category "World news as 0" and "Travel as 1"

In [ ]:
possible_category = mydata.category.unique()

label_dict = {}
for index, possible_categor in enumerate(possible_category):
    label_dict[possible_categor] = index
label_dict


## Vectorizing the category id and category to be used as target while running classifiers

In [ ]:
mydata['c_id'] = mydata.category.replace(label_dict)
mydata['category'] = mydata.category.replace(label_dict)

In [ ]:
mydata

### Splitting the dataset into X and y where X contains headline,authors,link,short description,date and category id

In [ ]:
X = mydata.iloc[:,2:8]
X.columns

### Setting y as a target("category")

In [ ]:
y = mydata['category']
print(y)

## 4.i) Specifying the size of the dataset and spliting the dataset into combined train plus valid datasets with size 0.75 and a test dataset with size 0.25

In [ ]:
X_train_plus_valid, X_test, y_train_plus_valid, y_test = train_test_split(X, y, random_state=0, test_size = 0.25, train_size = 0.75)


### 4.i)Then again splitting the combined train plus valid dataset into separate train with size 0.5 and valid datasets as 0.25

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_plus_valid, y_train_plus_valid, random_state=0, test_size = 0.25/0.75, train_size = 0.5/0.75)

### 4.ii) Storing the dataset into Excel csv files for train,test and valid datasets

In [ ]:
X_train.to_csv('train.csv',index=False)
X_test.to_csv('test.csv',index=False)
X_valid.to_csv('valid.csv',index=False)


### 5.) Loading the train dataset

In [ ]:
train=pd.read_csv('train.csv')
train

Removing every columns other than short description to run classification models

In [ ]:
del train['authors']
del train['link']
del train['headline']
del train['date']

Loading the train dataset

In [ ]:
train

### Preprocessing the train dataset like removing tags, special character and stop words to remove noise in the train dataset

In [ ]:
def remove_tags(text):
  remove = re.compile(r'')
  return re.sub(remove, '', text)
train['short_description'] = train['short_description'].apply(remove_tags)

def special_char(text):
  reviews = ''
  for x in text:
    if x.isalnum():
      reviews = reviews + x
    else:
      reviews = reviews + ' '
  return reviews
train['short_description'] = train['short_description'].apply(special_char)

def remove_stopwords(text):
  stop_words = set(stopwords.words('english'))
  words = word_tokenize(text)
  return " ".join([x for x in words if x not in stop_words])
train['short_description'] = train['short_description'].apply(remove_stopwords)

train

Vectorizing the Short description of train dataset using term frequency vectorizer


In [ ]:
tfidf_vecttrain = TfidfVectorizer(analyzer='word', max_features=6000)

x_tfidftrain = tfidf_vecttrain.fit_transform(train['short_description'])

x_featurestrain = pd.DataFrame(x_tfidftrain.toarray())

Splitiing the train dataset into train and test to run classifiers

In [ ]:
x_train, x_test, y_traint, y_testt = train_test_split(x_featurestrain, train['c_id'], test_size=.2)

## 6. Decision Tree classifier,Random forest classifier,Logistic Regression,Multinomial NB, K-Nearest neighbour and Linear SVC classifiers were used on train dataset.

Saving the model using decision tree classifier on train dataset

In [ ]:
#dtctrain = DecisionTreeClassifier(max_depth=500)

#dt_modeltrain = dtctrain.fit(x_train, y_traint)

# preddtctrain = dt_modeltrain.predict(x_test)

# print('Accuracy score :- {}'.format(round(accuracy_score(y_testt, preddtctrain), 2)))

# print(format(classification_report(y_testt, preddtctrain,zero_division=1)))

dt_modeltrain = DecisionTreeClassifier().fit(x_train, y_traint)
mymodel = open('dtctrain1.sav', 'wb')
pickle.dump( dt_modeltrain, mymodel)
mymodel.close()
preddtctrain = dt_modeltrain.predict(x_test)

In [ ]:
filename = 'dtctrain1.sav'
saved_clf = pickle.load(open(filename, 'rb'))
saved_clf.score(x_test, y_testt)

In [ ]:
print("---Test Set Results---")
print("Accuracy with logreg: {}".format(accuracy_score(y_testt, preddtctrain)))
print(classification_report(y_testt, preddtctrain))

## Task 3

### 8. Confusion matrix is a primary metric used to evaluate my models for summarizing the performance of a classification algorithm. The number of correct and incorrect predictions are summarized with count values and broken down by each class

In [ ]:
cm=confusion_matrix(y_testt, preddtctrain) #you will need to change the latter to your desired output
cm
 

ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax);  #Don't worry too much about the arguments here - we will return to this! 

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix');
ax.xaxis.set_ticklabels(['<=50K', '>50K']); ax.yaxis.set_ticklabels(['<=50K','>50K' ]);
TN, FP, FN, TP = confusion_matrix(y_testt, preddtctrain).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)
print('Accuracy score :- {}'.format(round(accuracy_score(y_testt, preddtctrain), 2)))

Storing Predicted values into a dataframe

In [ ]:
dtcpreda = pd.DataFrame(preddtctrain)
dtcpreda.columns = ["DTC"]

Running random forest classifier on train dataset

# 9.  The most accurate classifiers are decision tree classifier which has 0.91 and random forest classifier has the most accuracy score of 0.87 when compared to linear regression, multinomial nb and linear svc on train dataset and saving those models using pickle

In [ ]:
# rfctrain = RandomForestClassifier()

# rfc_modeltrain = rfctrain.fit(x_train, y_traint)

# rfcpredtrain = rfc_modeltrain.predict(x_test)

# print('Accuracy score :- {}'.format(round(accuracy_score(y_testt, rfcpredtrain), 2)))

# print(format(classification_report(y_testt, rfcpredtrain,zero_division=1)))
rfc_modeltrain = RandomForestClassifier().fit(x_train, y_traint)
mymodel = open('rtctrain1.sav', 'wb')
pickle.dump( rfc_modeltrain, mymodel)
mymodel.close()
rfcpredtrain = rfc_modeltrain.predict(x_test)

In [ ]:
filename = 'rtctrain1.sav'
saved_clf = pickle.load(open(filename, 'rb'))
saved_clf.score(x_test, y_testt)

### 10) Running classification report and confusion matrix for error analysis providing true positive,false positive,true negative,false negative and f-1 scores, precision, recall and support

In [ ]:
print("---Test Set Results---")
print("Accuracy with logreg: {}".format(accuracy_score(y_testt, rfcpredtrain)))
print(classification_report(y_testt, rfcpredtrain))

Running Confusion matrix for error analysis on train dataset for random forest classifier

In [ ]:
cm=confusion_matrix(y_testt, rfcpredtrain) #you will need to change the latter to your desired output
cm
 

ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax);  #Don't worry too much about the arguments here - we will return to this! 

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix');
ax.xaxis.set_ticklabels(['<=50K', '>50K']); ax.yaxis.set_ticklabels(['<=50K','>50K' ]);
TN, FP, FN, TP = confusion_matrix(y_testt, rfcpredtrain).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)
print('Accuracy score :- {}'.format(round(accuracy_score(y_testt, rfcpredtrain), 2)))

Storing Predicted values into a dataframe

In [ ]:
rfcpredaa = pd.DataFrame(rfcpredtrain)
rfcpredaa.columns = ["RFC"]

Predicted values after using classifiers on train dataset

In [ ]:
#https://softhints.com/display-two-pandas-dataframes-side-by-side-jupyter-notebook/
df3_styler = dtcpreda.style.set_table_attributes("style='display:inline'")
df4_styler = rfcpredaa.style.set_table_attributes("style='display:inline'")
display_html(df4_styler._repr_html_()+df3_styler._repr_html_(), raw=True)

Loading the valid dataset

In [ ]:
valid=pd.read_csv('valid.csv')
valid


Removing every columns other than short description 

In [ ]:
del valid['authors']
del valid['link']
del valid['headline']
del valid['date']

Preprocessing the valid dataset like removing tags, special character and stop words

In [ ]:
def remove_tags(text):
  remove = re.compile(r'')
  return re.sub(remove, '', text)
valid['short_description'] = valid['short_description'].apply(remove_tags)

def special_char(text):
  reviews = ''
  for x in text:
    if x.isalnum():
      reviews = reviews + x
    else:
      reviews = reviews + ' '
  return reviews
valid['short_description'] = valid['short_description'].apply(special_char)

def remove_stopwords(text):
  stop_words = set(stopwords.words('english'))
  words = word_tokenize(text)
  return " ".join([x for x in words if x not in stop_words])
valid['short_description'] = valid['short_description'].apply(remove_stopwords)

valid

Vectorizing the Short description of valid dataset 



In [ ]:
tfidf_vectvalid = TfidfVectorizer(analyzer='word', max_features=6000)

x_tfidfvalid = tfidf_vectvalid.fit_transform(valid['short_description'])

x_featuresvalid = pd.DataFrame(x_tfidfvalid.toarray())

Splitiing the valid dataset into train and test to run classifiers

In [ ]:
x_train1, x_test1, y_train1, y_test1 = train_test_split(x_featuresvalid, valid['c_id'], test_size=.2)

Decision Tree classifier,Random forest classifier,Logistic Regression,Multinomial NB, K-Nearest neighbour and Linear SVC classifiers were used on valid dataset and the most accurate ones are saved below.

Running decision tree classifier on valid dataset

In [ ]:
# dtcvalid = DecisionTreeClassifier(max_depth=500)

# dtcvalid_model = dtcvalid.fit(x_train1, y_train1)

# validdtcpred = dtcvalid_model.predict(x_test1)

# print('Accuracy score :- {}'.format(round(accuracy_score(y_test1, validdtcpred), 2)))

# print(format(classification_report(y_test1, validdtcpred,zero_division=1)))

dtcvalid_model = DecisionTreeClassifier().fit(x_train1, y_train1)
mymodel = open('dtctrain1.sav', 'wb')
pickle.dump( dtcvalid_model, mymodel)
mymodel.close()
validdtcpred = dtcvalid_model.predict(x_test1)

In [ ]:
filename = 'dtctrain1.sav'
saved_clf = pickle.load(open(filename, 'rb'))
saved_clf.score(x_test1, y_test1)

In [ ]:
print("---Test Set Results---")
print("Accuracy with logreg: {}".format(accuracy_score(y_test1, validdtcpred)))
print(classification_report(y_test1, validdtcpred))

Running Confusion matrix for error analysis on valid dataset for decision tree classifier

In [ ]:
cm=confusion_matrix(y_test1, validdtcpred) #you will need to change the latter to your desired output
cm
 

ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax);  #Don't worry too much about the arguments here - we will return to this! 

 #labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix');
ax.xaxis.set_ticklabels(['<=50K', '>50K']); ax.yaxis.set_ticklabels(['<=50K','>50K' ]);
TN, FP, FN, TP = confusion_matrix(y_test1, validdtcpred).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

print('Accuracy score :- {}'.format(round(accuracy_score(y_test1, validdtcpred), 2)))

Storing Predicted values into a dataframe

In [ ]:
dtcvalidpred = pd.DataFrame(validdtcpred)
dtcvalidpred.columns = ["Decision Tree Classifier"]


### 9. Accuracy scores of the valid datasets were 0.82 for the decision tree classifer and 0.8 using the random forest classifier are used while other classifiers had less accuracy scores. Accuracy scores of valid is less than the train dataset since the valid dataset has very less data when compared to train

Running random forest classifier on valid dataset

In [ ]:
# rfcvalid = RandomForestClassifier()

# rfcv_model = rfcvalid.fit(x_train1, y_train1)

# rfcpredvalid = rfcv_model.predict(x_test1)

# print('Accuracy score :- {}'.format(round(accuracy_score(y_test1, rfcpredvalid), 2)))

# print(format(classification_report(y_test1, rfcpredvalid,zero_division=1)))
rfcv_model = RandomForestClassifier().fit(x_train1, y_train1)
mymodel = open('rtctrain1.sav', 'wb')
pickle.dump( rfcv_model, mymodel)
mymodel.close()
rfcpredvalid = rfcv_model.predict(x_test1)

In [ ]:
filename = 'rtctrain1.sav'
saved_clf = pickle.load(open(filename, 'rb'))
saved_clf.score(x_test1, y_test1)

In [ ]:
print("---Test Set Results---")
print("Accuracy with logreg: {}".format(accuracy_score(y_test1, rfcpredvalid)))
print(classification_report(y_test1, rfcpredvalid))

Running Confusion matrix for error analysis on valid dataset for random forest classifier

In [ ]:
cm=confusion_matrix(y_test1, rfcpredvalid) #you will need to change the latter to your desired output
cm
 

ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax);  #Don't worry too much about the arguments here - we will return to this! 

 #labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix');
ax.xaxis.set_ticklabels(['<=50K', '>50K']); ax.yaxis.set_ticklabels(['<=50K','>50K' ]);
TN, FP, FN, TP = confusion_matrix(y_test1, rfcpredvalid).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

print('Accuracy score :- {}'.format(round(accuracy_score(y_test1, rfcpredvalid), 2)))

Storing Predicted values into a dataframe

In [ ]:
rfcpredaav = pd.DataFrame(rfcpredvalid)
rfcpredaav.columns = ["RFC"]
rfcpredaav

Printing Predicted values after using classifiers on valid dataset

In [ ]:
#https://softhints.com/display-two-pandas-dataframes-side-by-side-jupyter-notebook/
df3_styler = dtcvalidpred.style.set_table_attributes("style='display:inline'").set_caption('df3')
df4_styler = rfcpredaav.style.set_table_attributes("style='display:inline'").set_caption('df4')
display_html(df4_styler._repr_html_()+df3_styler._repr_html_(), raw=True)

# 11.)Adding a preprocessing technique to train dataset and using classifiers

Making a copy of train data set into trainmod dataset to use classifiers

In [ ]:
traina = pd.DataFrame(train)

#Make a copy:

trainmod = traina.copy()

print(trainmod)

### Adding a preprocessing technique lemmatization to the train modification dataset

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
#code reference: stackoverflow.com
def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])
trainmod['short_description'] = trainmod.short_description.apply(lemmatize_text)
print(trainmod)

Vectorizing the Short description of train modified dataset 


In [ ]:
tfidf_vectt_modifiedtrain = TfidfVectorizer(analyzer='word', max_features=6000)

x_tfidfft_modifiedtrain = tfidf_vectt_modifiedtrain.fit_transform(trainmod['short_description'])

x_featuress_modifiedtrain= pd.DataFrame(x_tfidfft_modifiedtrain .toarray())

Splitiing the train modified dataset into train and test to run classifiers

In [ ]:
x_train_m, x_test_m, y_train_m, y_test_m = train_test_split(x_featuress_modifiedtrain, trainmod['c_id'], test_size=.2)

Decision Tree classifier,Random forest classifier,Logistic Regression,Multinomial NB, K-Nearest neighbour and Linear SVC classifiers were used on train modified dataset and the most accurate ones are saved below.

Running decision tree classifier on train modified dataset

In [ ]:
# dtc = DecisionTreeClassifier(max_depth=500)

# dt_model = dtc.fit(x_train_m, y_train_m)

# preddtc_m = dt_model.predict(x_test_m)

# print('Accuracy score :- {}'.format(round(accuracy_score(y_test_m, preddtc_m), 2)))

# print(format(classification_report(y_test_m, preddtc_m,zero_division=1)))
dt_model = DecisionTreeClassifier().fit(x_train_m, y_train_m)
mymodel = open('dtctrainmod.sav', 'wb')
pickle.dump( dt_model, mymodel)
mymodel.close()
preddtc_m = dt_model.predict(x_test_m)

In [ ]:
filename = 'dtctrainmod.sav'
saved_clf = pickle.load(open(filename, 'rb'))
saved_clf.score(x_test_m, y_test_m)

In [ ]:
print("---Test Set Results---")
print("Accuracy with logreg: {}".format(accuracy_score(y_test_m, preddtc_m)))
print(classification_report(y_test_m, preddtc_m))

Storing Predicted values of train modified into a dataframe

In [ ]:
dtcpreda = pd.DataFrame(preddtc_m)
dtcpreda.columns = ["DTC"]
dtcpreda

Running Confusion matrix for error analysis on train modified dataset for decision tree classifier

In [ ]:
cm=confusion_matrix(y_test_m, preddtc_m) #you will need to change the latter to your desired output
cm
 
ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax);  #Don't worry too much about the arguments here - we will return to this! 

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix');
#ax.xaxis.set_ticklabels(['<=50K', '>50K']); ax.yaxis.set_ticklabels(['<=50K','>50K' ]);
TN, FP, FN, TP = confusion_matrix(y_test_m, preddtc_m).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

print('Accuracy score :- {}'.format(round(accuracy_score(y_test_m, preddtc_m), 2)))

Running random forest classifier on train modified dataset and saved to a model

# 11) After lemmatization, a modification in the train set has the performance accuracy score in random forest classifier better then the train dataset and saving the model below using pickle 

In [ ]:
rfctrainmod = RandomForestClassifier().fit(x_train_m, y_train_m)
mymodel = open('rtctrainmod.sav', 'wb')
pickle.dump(rfctrainmod, mymodel)
mymodel.close()

Loading the model using pickle 

In [ ]:
filename = 'rtctrainmod.sav'
saved_clf = pickle.load(open(filename, 'rb'))
saved_clf.score(x_test_m, y_test_m)

In [ ]:
y_preCIONNNN = rfctrainmod.predict(x_test_m)
print("---Test Set Results---")
print("Accuracy with logreg: {}".format(accuracy_score(y_test_m, y_preCIONNNN)))
print(classification_report(y_test_m, y_preCIONNNN))

Running Confusion matrix for error analysis on train modified dataset for random forest classifier

In [ ]:

cm=confusion_matrix(y_test_m, y_preCIONNNN) #you will need to change the latter to your desired output
cm
 
ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax);  #Don't worry too much about the arguments here - we will return to this! 

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix');
#ax.xaxis.set_ticklabels(['<=50K', '>50K']); ax.yaxis.set_ticklabels(['<=50K','>50K' ]);
TN, FP, FN, TP = confusion_matrix(y_test_m, y_preCIONNNN).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

print('Accuracy score :- {}'.format(round(accuracy_score(y_test_m, y_preCIONNNN), 2)))

Storing Predicted values of train modified into a dataframe

In [ ]:
rfcpredaa = pd.DataFrame(y_preCIONNNN)
rfcpredaa.columns = ["rfc"]
rfcpredaa

Printing Predicted values after using classifiers on valid modified dataset

In [ ]:
df3_styler = dtcpreda.style.set_table_attributes("style='display:inline'").set_caption('df3')
df4_styler = rfcpredaa.style.set_table_attributes("style='display:inline'").set_caption('df4')


display_html(df4_styler._repr_html_()+df3_styler._repr_html_(), raw=True)


#https://softhints.com/display-two-pandas-dataframes-side-by-side-jupyter-notebook/

Making a copy of valid dataframe to modify and apply classifiers

In [ ]:
valida = pd.DataFrame(valid)

#Make a copy:

validmod = valida.copy()

print(validmod)

#### Adding a preprocessing technique lemmatization to the valid modification dataset

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])


validmod['short_description'] = validmod.short_description.apply(lemmatize_text)

The valid modified dataset is now vectorized using term frequency vectorizer and then split into train and test for running classifers such as KNN,Decision Tree,Random Forest,Multinomial NB and Linear SVC. The Models with greater accuracy scores are saved and predicted values are printed

In [ ]:
tfidf_vectv_validmod = TfidfVectorizer(analyzer='word', max_features=6000)

x_tfidfvalidm = tfidf_vectv_validmod.fit_transform(validmod['short_description'])

x_features_validm = pd.DataFrame(x_tfidfvalidm.toarray())

In [ ]:
x_validm, x_test_vm, y_validm, y_test_vm = train_test_split(x_features_validm, validmod['c_id'], test_size=.2)

In [ ]:
# dtcv = DecisionTreeClassifier(max_depth=500)

# dt_modelv = dtcv.fit(x_validm, y_validm)

# preddtcv = dt_modelv.predict(x_test_vm)

# print('Accuracy score :- {}'.format(round(accuracy_score(y_test_vm, preddtcv), 2)))

# print(format(classification_report(y_test_vm, preddtcv,zero_division=1)))
dt_modelv = DecisionTreeClassifier().fit(x_validm, y_validm)
mymodel = open('dtctrainmod.sav', 'wb')
pickle.dump( dt_modelv, mymodel)
mymodel.close()
preddtcv = dt_modelv.predict(x_test_vm)

In [ ]:
filename = 'dtctrainmod.sav'
saved_clf = pickle.load(open(filename, 'rb'))
saved_clf.score(x_test_vm, y_test_vm)

In [ ]:

print("---Test Set Results---")
print("Accuracy with logreg: {}".format(accuracy_score(y_test_vm, preddtcv)))
print(classification_report(y_test_vm, preddtcv))

## After lemmatization on valid dataset, the decision tree classifier of valid dataset the accuracy scores improved to 0.81 from 0.83 

In [ ]:
cm=confusion_matrix(y_test_vm, preddtcv) #you will need to change the latter to your desired output
cm
 
ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax);  #Don't worry too much about the arguments here - we will return to this! 

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix');
#ax.xaxis.set_ticklabels(['<=50K', '>50K']); ax.yaxis.set_ticklabels(['<=50K','>50K' ]);
TN, FP, FN, TP = confusion_matrix(y_test_vm, preddtcv).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

print('Accuracy score :- {}'.format(round(accuracy_score(y_test_vm, preddtcv), 2)))

In [ ]:
dtcpreda = pd.DataFrame(preddtcv)
dtcpreda.columns = ["DTC"]
dtcpreda

In [ ]:
# rfcv = RandomForestClassifier()

# rfc_modelv = rfcv.fit(x_validm, y_validm)

# rfcpredav = rfc_modelv.predict(x_test_vm)

# print('Accuracy score :- {}'.format(round(accuracy_score(y_test_vm, rfcpredav), 2)))

# print(format(classification_report(y_test_vm, rfcpredav,zero_division=1)))#
rfc_modelv = RandomForestClassifier().fit(x_validm, y_validm)
mymodel = open('rtctrainmod.sav', 'wb')
pickle.dump( rfc_modelv, mymodel)
mymodel.close()
rfcpredav = rfc_modelv.predict(x_test_vm)

In [ ]:
filename = 'rtctrainmod.sav'
saved_clf = pickle.load(open(filename, 'rb'))
saved_clf.score(x_test_vm, y_test_vm)

In [ ]:
print("---Test Set Results---")
print("Accuracy with logreg: {}".format(accuracy_score(y_test_vm, rfcpredav)))
print(classification_report(y_test_vm, rfcpredav))

## After lemmatization on valid dataset, the random forest classifier of valid dataset the accuracy scored improved to 0.82 from 0.81 

In [ ]:
cm=confusion_matrix(y_test_vm, rfcpredav) #you will need to change the latter to your desired output
cm
 
ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax);  #Don't worry too much about the arguments here - we will return to this! 

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix');
#ax.xaxis.set_ticklabels(['<=50K', '>50K']); ax.yaxis.set_ticklabels(['<=50K','>50K' ]);
TN, FP, FN, TP = confusion_matrix(y_test_vm, rfcpredav).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

print('Accuracy score :- {}'.format(round(accuracy_score(y_test_vm, rfcpredav), 2)))

In [ ]:
rfcpredaa = pd.DataFrame(rfcpredav)
rfcpredaa.columns = ["rfc"]
rfcpredaa

In [ ]:
df3_styler = dtcpreda.style.set_table_attributes("style='display:inline'").set_caption('df3')
df4_styler = rfcpredaa.style.set_table_attributes("style='display:inline'").set_caption('df4')
display_html(df4_styler._repr_html_()+df3_styler._repr_html_(), raw=True)

#https://softhints.com/display-two-pandas-dataframes-side-by-side-jupyter-notebook/

# 12)Merging the Train and valid dataset into one dataset and performing preprocessing

In [ ]:
a_X = pd.concat([X_train, X_valid], axis=0)

In [ ]:
a_X

In [ ]:
a_Y = pd.concat([y_train, y_valid], axis=0)

In [ ]:
a_Y

Preprocessing the merged dataset by removing stopwords, lemmatizing and removing other columns rather than short description

In [ ]:
def remove_stopwords(text):
  stop_words = set(stopwords.words('english'))
  words = word_tokenize(text)
  return " ".join([x for x in words if x not in stop_words])
a_X['short_description'] = a_X['short_description'].apply(remove_stopwords)
a_X

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])


a_X['short_description'] = a_X.short_description.apply(lemmatize_text)

In [ ]:
del a_X['authors']
del a_X['link']
del a_X['headline']
del a_X['date']

In [ ]:
del a_X['c_id']

In [ ]:
a_X

In [ ]:
def special_char(text):
  reviews = ''
  for x in text:
    if x.isalnum():
      reviews = reviews + x
    else:
      reviews = reviews + ' '
  return reviews
a_X['short_description'] = a_X['short_description'].apply(special_char)
a_X

Vectorizing the short description of merged train and valid dataset using term frequency vectorizer

In [ ]:
tfidf_vectt_merged = TfidfVectorizer(analyzer='word', max_features=6000)

x_tfidfft_merged = tfidf_vectt_merged.fit_transform(a_X['short_description'])

x_featuress_tmerged= pd.DataFrame(x_tfidfft_merged .toarray())

Initialising our KFold object with 5 splits to run K-fold Cross Validation on merged train and valid dataset

In [ ]:
folder = KFold(n_splits=5)

In [ ]:
results = cross_val_score(dt_modelv , x_featuress_tmerged, a_Y, cv = folder)# Parameters are, in order - model, X data, labels (y), and the KFold object
results

In [ ]:
output = df(results, columns=["Accuracy_per_fold"])
output

Accuracy of K-Fold Cross Validation using Decision Tree Classifier

In [ ]:
# the mean of the column!
av_val = output["Accuracy_per_fold"].mean()*100
print("The average accuracy across folds is {}%".format(av_val))

Initialising our KFold object with 5 splits to run K-fold Cross Validation on merged train and valid dataset

In [ ]:
results1 = cross_val_score(rfc_modelv , x_featuress_tmerged, a_Y, cv = folder)# Parameters are, in order - model, X data, labels (y), and the KFold object
results1

In [ ]:
output1 = df(results1, columns=["Accuracy_per_fold"])
output1

Accuracy of K-Fold Cross Validation using Random Forest Classifier

In [ ]:
#get the mean of the column!
av_val = output1["Accuracy_per_fold"].mean()*100
print("The average accuracy across folds is {}%".format(av_val))

### After doing cross validating using K-Fold using Decision Tree classifier and Random forest classiffier. Accuracy scores of Random Forest classifier were greater which is 87.6948 than decision tree classifier which is 82.24

## 13. Choosing the random forest classifier to run on test dataset after doing modification in the train dataset which has the best accuracy scores from 11)

Loading the test dataset 

In [ ]:
testtt=pd.read_csv('test.csv')

In [ ]:
testtt

Removing all the column rather than short description and category in Test dataset

In [ ]:
del testtt['authors']
del testtt['link']
del testtt['headline']
del testtt['date']

Preprocessing techniques like removing tags,special characters,stop words and lemmatizing the test dataset

In [ ]:
def remove_tags(text):
  remove = re.compile(r'')
  return re.sub(remove, '', text)
testtt['short_description'] = testtt['short_description'].apply(remove_tags)

def special_char(text):
  reviews = ''
  for x in text:
    if x.isalnum():
      reviews = reviews + x
    else:
      reviews = reviews + ' '
  return reviews
testtt['short_description'] = testtt['short_description'].apply(special_char)

def remove_stopwords(text):
  stop_words = set(stopwords.words('english'))
  words = word_tokenize(text)
  return " ".join([x for x in words if x not in stop_words])
testtt['short_description'] = testtt['short_description'].apply(remove_stopwords)

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])


testtt['short_description'] = testtt.short_description.apply(lemmatize_text)
testtt

Vectorizing using Term frequency vectorizer on test dataset for short description

In [ ]:
tfidf_test= TfidfVectorizer(analyzer='word', max_features=6000)

x_tfidftest= tfidf_test.fit_transform(testtt['short_description'])

x_featuresxtest = pd.DataFrame(x_tfidftest.toarray())

Splitiing the test dataset into train and test to use classifiers

In [ ]:
x_traintest, x_testtest, y_traintest, y_testtest = train_test_split(x_featuresxtest, testtt['c_id'], test_size=.2)

In [ ]:
rfcmerged_model = RandomForestClassifier().fit(x_traintest, y_traintest)
mymodel = open('rtctrainmod.sav', 'wb')
pickle.dump( rfcmerged_model, mymodel)
mymodel.close()
rfcpredav = rfcmerged_model.predict(x_testtest)

### Loading the best accuracy random forest classifier model saved from train modified dataset into test dataset

In [ ]:
filename = 'rtctrainmod.sav'
saved_clf = pickle.load(open(filename, 'rb'))
saved_clf.score(x_testtest, y_testtest)

The accuracy obtained in test model using the saved model from modified train dataset is 0.90 which has increased previously from valid modified 

Accuracy classification report with error analysis

In [ ]:
print("---Test Set Results---")
print("Accuracy with logreg: {}".format(accuracy_score(y_testtest, rfcpredav)))
print(classification_report(y_testtest, rfcpredav))

Splitiing themerged dataset train and valid merged dataset to run classifiers

In [ ]:
x_train_mergedd, x_test_merged, y_train_merged, y_test_merged = train_test_split(x_featuress_tmerged, a_Y, test_size=0.5)

## Running random forest classifier on merged dataset as it gave the best accuracy score on cross validation of merged dataset

In [ ]:
# rfcmergedd = RandomForestClassifier()

# rfcmerged_model = rfcmergedd.fit(x_train_mergedd, y_train_merged)

# rfcmergepred = rfcmerged_model.predict(x_test_merged)

# print('Accuracy score :- {}'.format(round(accuracy_score(y_test_merged, rfcmergepred), 2)))
# print("Train data accuracy:",accuracy_score(y_train_merged,rfcmergepred)) 

# print(format(classification_report(y_test_merged, rfcmergepred,zero_division=1)))
rfcmerged_model = RandomForestClassifier().fit(x_train_mergedd, y_train_merged)
mymodel = open('rtctrainmod.sav', 'wb')
pickle.dump( rfcmerged_model, mymodel)
mymodel.close()
rfcpredav = rfcmerged_model.predict(x_test_merged)

In [ ]:
filename = 'rtctrainmod.sav'
saved_clf = pickle.load(open(filename, 'rb'))
saved_clf.score(x_test_merged, y_test_merged)

In [ ]:
print("---Test Set Results---")
print("Accuracy with logreg: {}".format(accuracy_score(y_test_merged, rfcpredav)))
print(classification_report(y_test_merged, rfcpredav))

Using the saved random forest classifier model of merged dataset on test dataset

In [ ]:
filename = 'rtctrainmod.sav'
saved_clfa = pickle.load(open(filename, 'rb'))
saved_clfa.score(x_testtest, y_testtest)


In [ ]:
y_predictionmodela = rfcmerged_model.predict(x_testtest)
print("---Test Set Results---")
print("Accuracy with logreg: {}".format(accuracy_score(y_testtest, y_predictionmodela)))
print(classification_report(y_testtest, y_predictionmodela))

Accuracy of the test model after retrained using merged dataset is 0.73

## 7)Deep Learning from the scratch-- Taking the copy of whole dataset

In [ ]:
# code reference: github.com 

copya = pd.DataFrame(mydata)

#Make a copy:

hypertuning = copya.copy()

print(hypertuning)

Reducing the length of rows to 1000 to hypertune and regualrize the small dataset

In [ ]:
hypertuning=hypertuning[0:1000]
len(hypertuning)

Merging the headline and short description into sentence

In [ ]:
hypertuning["sentence"]=hypertuning['headline']+hypertuning['short_description']
hypertuning["category"]=hypertuning['category']

removing all other columns rather than sentence and category

In [ ]:
del hypertuning['authors']
del hypertuning['link']
del hypertuning['headline']
del hypertuning['date']
del hypertuning['id']
del hypertuning['short_description']
del hypertuning['c_id']

hypertuning


building vocabulary from dataset

In [ ]:
time_steps = 100

 
def build_vocabulary(sentence_list):
    unique_words = " ".join(sentence_list).strip().split()
    word_count = Counter(unique_words).most_common()
    vocabulary = {}
    for word, _ in word_count:
        vocabulary[word] = len(vocabulary)        

    return vocabulary

vocabulary = build_vocabulary(hypertuning["sentence"])

In [ ]:
inputs = Input(shape=(500, ))
embedding_layer = Embedding(len(vocabulary),
                            16,input_length=500)(inputs)
word_embedding= BatchNormalization()(embedding_layer)

x = LSTM(128)(word_embedding)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=[inputs], outputs=predictions)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

model.summary()

Create datasets (Only take up to time_steps words for memory)

In [ ]:
train_text = hypertuning['sentence'].tolist()
train_text = [' '.join(t.split()[0:time_steps]) for t in train_text]
train_text = np.array(train_text)
train_label = np.array(hypertuning['category'].tolist())

Preprocessing the dataset

In [ ]:
MAX_LENGTH = 500
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_text)
post_seq = tokenizer.texts_to_sequences(train_text)
post_seq_padded = pad_sequences(post_seq, maxlen=MAX_LENGTH)

Splitting the dataset to hypertune

In [ ]:
X_trainhypertuning, X_testhypertuning, y_trainhypertuning, y_testhypertuning = train_test_split(post_seq_padded, train_label, test_size=0.5)

In [ ]:
history = model.fit([X_trainhypertuning], y=to_categorical(y_trainhypertuning), verbose=1, validation_split=0.25, 
          shuffle=False, epochs=5)

In [ ]:
model.evaluate(X_testhypertuning,to_categorical(y_testhypertuning))

### - To decrease overfitting in my simple model and for that aim I decreased my sample to 1000.
### - At the above we can see my base model there is huge overfitting like almost %30


## Adding Dropout Layer

In [ ]:
result = {}
y = {}
loss = []
acc = []
dropouts = [0.0, 0.1, 0.2, 0.3]
for dropout in dropouts:
    print ("Dropout: ", dropout)

    inputs = Input(shape=(500, ))
    embedding_layer = Embedding(len(vocabulary),
                            16,input_length=500)(inputs)
    word_embedding= BatchNormalization()(embedding_layer)

    x = LSTM(128)(word_embedding)
    x = Dense(64, activation='relu')(x)
    x = Dropout(dropout)(x)
    x = Dense(32, activation='relu')(x)
    predictions = Dense(2, activation='softmax')(x)
    model = Model(inputs=[inputs], outputs=predictions)
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
    model_simple = model.fit([X_trainhypertuning], batch_size=64, y=to_categorical(y_trainhypertuning), verbose=1, validation_split=0.25, 
          shuffle=False, epochs=5)
    score = model.evaluate(X_testhypertuning, to_categorical(y_testhypertuning),verbose=0)
    y[dropout] = model.predict(X_testhypertuning)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])
    result[dropout] = copy.deepcopy(model_simple.history)   
    loss.append(score[0])
    acc.append(score[1])

### As we can see dropout with rate 0.1 helped us do decrease overfitting to %25

In [ ]:
width = 0.1

plt.bar(dropouts, acc, width, align='center')
#plt.figure(figsize=(50,50))
plt.tick_params(axis='both', which='major', labelsize=10)
plt.tick_params(axis='both', which='minor', labelsize=10)

plt.ylabel('Accuracy',size = 20)
plt.xlabel('Dropout', size = 20)
plt.show()

In [ ]:
width = 0.1

plt.bar(dropouts, loss, width, align='center',color = 'green')

plt.tick_params(axis='both', which='major', labelsize=10)
plt.tick_params(axis='both', which='minor', labelsize=10)

plt.ylabel('Loss',size = 20)
plt.xlabel('Dropout', size = 20)
plt.show()

fine tuning hyper parameters

In [ ]:
result = {}
y = {}
loss = []
acc = []

learning_rates = [0.1,0.01,0.001]
lr_weight_decay = 0.95  

for lr in learning_rates:
    print ("Learning rate: ", lr)
    
    inputs = Input(shape=(500, ))
    embedding_layer = Embedding(len(vocabulary),
                            16,input_length=500)(inputs)
    word_embedding= BatchNormalization()(embedding_layer)

    x = LSTM(128)(word_embedding)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(32, activation='relu')(x)
    predictions = Dense(2, activation='softmax')(x)
    model = Model(inputs=[inputs], outputs=predictions)
    adam = optimizers.Adam(lr,decay=lr_weight_decay)
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['acc'])
    
    model_simple = model.fit([X_trainhypertuning], batch_size=64, y=to_categorical(y_trainhypertuning), verbose=1, validation_split=0.25, 
          shuffle=False, epochs=5)
    score = model.evaluate(X_testhypertuning, to_categorical(y_testhypertuning),verbose=0)
    y[lr] = model.predict(X_testhypertuning)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])
    result[lr] = copy.deepcopy(model_simple.history)   
    loss.append(score[0])
    acc.append(score[1])

In [ ]:
width = 0.1

plt.bar(learning_rates, acc, width, align='center')
#plt.figure(figsize=(50,50))
plt.tick_params(axis='both', which='major', labelsize=20)
plt.tick_params(axis='both', which='minor', labelsize=20)

plt.ylabel('Accuracy',size = 20)
plt.xlabel('Learning Rate', size = 20)
plt.show()

In [ ]:
width = 0.1

plt.bar(learning_rates, loss, width, align='center',color = 'green')

plt.tick_params(axis='both', which='major', labelsize=10)
plt.tick_params(axis='both', which='minor', labelsize=10)

plt.ylabel('Loss',size = 20)
plt.xlabel('Learning rates', size = 20)
plt.show()

## So also we can try different optimizer as RMSPROP

In [ ]:
inputs = Input(shape=(500, ))
embedding_layer = Embedding(len(vocabulary),
                            16,input_length=500)(inputs)
word_embedding= BatchNormalization()(embedding_layer)

x = LSTM(128)(word_embedding)
x = Dense(64, activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(32, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=[inputs], outputs=predictions)
rmsprop = optimizers.RMSprop(learning_rate=0.001, decay=0.95)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

model.summary()

In [ ]:
model_simple = model.fit([X_trainhypertuning], batch_size=64, y=to_categorical(y_trainhypertuning), verbose=1, validation_split=0.25, 
          shuffle=False, epochs=5)

In [ ]:

predicted=model.predict(X_testhypertuning)
predicted = np.argmax(predicted, axis=1)
print(accuracy_score(y_testhypertuning, predicted))

As we can see from above results we succeded to decrease overfitting to %25 

## As a result of these experiments I can say that Dropout , changing optimizer to rmsprop helped us to acvhieve better performance so now we can try these changes on our whole dataset

Taking a copy of the whole dataframe to optimize using rmsprop

In [ ]:
copya = pd.DataFrame(mydata)

#Make a copy:

hypertuninga = copya.copy()

print(hypertuninga)

Merging the headline and short descsription to a sentence column

In [ ]:
hypertuninga["sentence"]=hypertuninga['headline']+hypertuninga['short_description']
hypertuninga["category"]=hypertuninga['category']

Removing all the columns rather than category and sentence

In [ ]:
del hypertuninga['authors']
del hypertuninga['link']
del hypertuninga['headline']
del hypertuninga['date']
del hypertuninga['id']
del hypertuninga['short_description']
del hypertuninga['c_id']

hypertuninga


Building a vocabulary

In [ ]:
vocabulary = build_vocabulary(hypertuninga["sentence"])

Create datasets 

In [ ]:
train_text = hypertuninga['sentence'].tolist()
train_text = [' '.join(t.split()[0:time_steps]) for t in train_text]
train_text = np.array(train_text)
train_label = np.array(hypertuninga['category'].tolist())

In [ ]:
MAX_LENGTH = 500
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_text)
post_seq = tokenizer.texts_to_sequences(train_text)
post_seq_padded = pad_sequences(post_seq, maxlen=MAX_LENGTH)

In [ ]:
X_trainhyper, X_testhyper, y_trainhyper, y_testhyper = train_test_split(post_seq_padded, train_label, test_size=0.5)

In [ ]:

inputs = Input(shape=(500, ))
embedding_layer = Embedding(len(vocabulary),
                            16,input_length=500)(inputs)
word_embedding= BatchNormalization()(embedding_layer)

x = LSTM(128)(word_embedding)
x = Dense(64, activation='relu',kernel_regularizer=regularizers.l2(0.05))(x)
x = Dropout(0.1)(x)
x = Dense(32, activation='relu',kernel_regularizer=regularizers.l2(0.05))(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=[inputs], outputs=predictions)
rmsprop = optimizers.RMSprop(learning_rate=0.001, decay=0.95)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

model.summary()

In [ ]:
history = model.fit([X_trainhyper], batch_size=128, y=to_categorical(y_trainhyper), verbose=1, validation_split=0.25, 
          shuffle=False, epochs=10)

# summarize history for accuracy

In [ ]:

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


# summarize history for loss

In [ ]:

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()